# Mini Project Income - Pipeline Interactivo
Este notebook te permite ejecutar paso a paso el flujo del proyecto, importando los módulos desde `src/`.

In [ ]:
import sys
sys.path.append("../")  # Asegura que Python encuentre los módulos en ../src

from src.config import config
from src import (
    load_validate,
    target_processing,
    cleaning,
    eda,
    feature_engineering,
    split,
    pipeline,
    experiment
)


## 1. Carga y Validación Inicial

In [ ]:
df = load_validate.load_dataset(config["path"])
df = load_validate.validate_structure(df, config["expected_columns"])
load_validate.quick_explore(df)
load_validate.check_nulls_duplicates(df)


## 2. Procesamiento del Target

In [ ]:
df = target_processing.clean_normalize_target(df, config["target"])
df = target_processing.encode_target(df, config["target"], config["target_mapping"])
target_processing.plot_target_distribution(df, config["target"])
removed_rows = df[config["target"]].isna().sum()
target_processing.log_target_status(df, config["target"], removed_rows)


## 3. Limpieza General

In [ ]:
df, _ = cleaning.drop_duplicates(df)
cleaning.analyze_missing_values(df, config["target"])
df = cleaning.fix_column_types_and_formats(df)
cleaning.detect_gross_outliers(df, config["num_features"])


## 4. Análisis Exploratorio de Datos (EDA)

In [ ]:
eda.describe_numerical(df, config["num_features"])
eda.describe_categorical(df, config["cat_features"])
eda.plot_numerical_distributions(df, config["num_features"])
eda.plot_categorical_distributions(df, config["cat_features"], min_freq=10)
eda.plot_feature_target_relationship(df, config["features"], config["target"])
corr = eda.correlation_analysis(df, config["num_features"])
eda.analyze_feature_redundancy(df, config["num_features"])


## 5. Ingeniería y Selección de Features

In [ ]:
features_dict = feature_engineering.prepare_final_features(df, config["drop_features"], config["target"])
feature_engineering.collect_pipeline_issues(df, features_dict)
feature_engineering.save_feature_config(features_dict, "features_config.json")


## 6. División de Datos

In [ ]:
X, y = split.prepare_X_y(df, features_dict, config["target"])
splits = split.stratified_train_test_split(X, y)
split.check_split_quality(splits["y_train"], splits["y_test"])


## 7. Pipeline + Experimento con MLflow

In [ ]:
pipe = pipeline.make_pipeline_for_experiment(config, features_dict)

param_grid = {
    "model__C": [0.1, 1, 10],
    "preprocessing__num__outlier__strategy": ["clip", "none"]
}

experiment.setup_mlflow_experiment(
    tracking_uri=config["mlflow"]["tracking_uri"],
    experiment_name=config["mlflow"]["experiment_name"]
)

experiment.run_gridsearch_experiment(
    pipe, param_grid,
    splits["X_train"], splits["y_train"],
    splits["X_test"], splits["y_test"],
    scoring="accuracy",
    out_dir="results_out",
    tags={"proyecto": "mini_project_income", "autor": "Javiercito"}
)
